<a href="https://colab.research.google.com/github/6058ah/BusinessCapstone/blob/master/%EC%A3%BC%EC%8B%9D%EB%8D%B0%EC%9D%B4%ED%84%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#회사명 및 종목 코드 목록 불러오기

In [162]:
import pandas as pd
import pandas_datareader as pdr

# 종목 타입에 따라 download url이 다름. 종목코드 뒤에 .KS .KQ등이 입력되어야해서 Download Link 구분 필요
stock_type = {
'kospi': 'stockMkt',
'kosdaq': 'kosdaqMkt'
}

# 회사명으로 주식 종목 코드를 획득할 수 있도록 하는 함수
def get_code(df, name):
  code = df.query("name=='{}'".format(name))['code'].to_string(index=False)
# 위와같이 code명을 가져오면 앞에 공백이 붙어있는 상황이 발생하여 앞뒤로 sript() 하여 공백 제거
  code = code.strip()
  return code


# download url 조합
def get_download_stock(market_type=None):
  market_type = stock_type[market_type]
  download_link = 'http://kind.krx.co.kr/corpgeneral/corpList.do'
  download_link = download_link + '?method=download'
  download_link = download_link + '&marketType=' + market_type

  df = pd.read_html(download_link, header=0)[0]
  return df;


# kospi 종목코드 목록 다운로드
def get_download_kospi():
  df = get_download_stock('kospi')
  df.종목코드 = df.종목코드.map('{:06d}.KS'.format)
  return df

# kospi종목코드 다운로드
kospi_df = get_download_kospi()


# data frame merge
code_df = pd.concat([kospi_df])


# data frame정리
code_df = code_df[['회사명', '종목코드']]


# data frame title 변경 '회사명' = name, 종목코드 = 'code'
code_df = code_df.rename(columns={'회사명': 'name', '종목코드': 'code'})

code_df

,name,code
0,DSR,155660.KS
1,GS글로벌,001250.KS
2,HDC현대산업개발,294870.KS
3,KEC,092220.KS
4,KG동부제철,016380.KS
...,...,...
800,현대위아,011210.KS
801,현대코퍼레이션,011760.KS
802,화신,010690.KS
803,휴켐스,069260.KS


#KOSPI 상위 20 종목 가져오기

In [163]:
from bs4 import BeautifulSoup
import requests

url = 'https://finance.naver.com/sise/sise_market_sum.nhn?sosok=0'
req = requests.get(url)
 
if req.ok:
  html = req.text
  soup = BeautifulSoup(html,'html.parser')

Top_KOSPI = soup.select('table > tbody > tr > td > a')
Top20_KOSPI = [each_line.get_text() for each_line in Top_KOSPI[:40]]
Top20_KOSPI = Top20_KOSPI[0::2]


# 현대차, POSCO로 야후 파이낸스에서 인식 못함
# 삼성전자우는 삼성전자와 동일 기업임으로 엔씨소프트와 교체
Top20_KOSPI[7] = '현대모비스'
Top20_KOSPI[12] = '포스코'
del Top20_KOSPI[3]
Top20_KOSPI.append('엔씨소프트')

#데이터 추출

In [169]:
#print(Top20_KOSPI)
code_KOSPI = []
for x in range(0,20):
  code = get_code(code_df, Top20_KOSPI[x])
  code_KOSPI.append(code)
#print(code_KOSPI)

['삼성전자', 'SK하이닉스', 'NAVER', 'LG화학', '삼성바이오로직스', '카카오', '현대모비스', '삼성SDI', '셀트리온', '기아', '현대모비스', '포스코', 'LG전자', 'LG생활건강', '삼성물산', 'SK텔레콤', 'SK이노베이션', 'KB금융', 'SK', '엔씨소프트']
['005930.KS', '000660.KS', '035420.KS', '051910.KS', '207940.KS', '035720.KS', '012330.KS', '006400.KS', '068270.KS', '000270.KS', '012330.KS', '005490.KS', '066570.KS', '051900.KS', '028260.KS', '017670.KS', '096770.KS', '105560.KS', '034730.KS', '036570.KS']


In [165]:
from datetime import datetime
start = datetime(2011,1,1)
end = datetime(2021,4,9)
# open 시가
# high 고가
# low 저가
# close 종가
# volume 거래량
# Adj Close 주식의 분할, 배당, 배분 등을 고려해 조정한 종가

In [170]:
writer = pd.ExcelWriter('KOSPI_TOP20.xlsx', engine='xlsxwriter')
for i in range(0,20):
  df = pdr.get_data_yahoo(code_KOSPI[i],start,end)
  df.to_excel(writer, sheet_name= Top20_KOSPI[i])

writer.save()